<h1>Diamond price prediction using Multiple Linear Regression </h1>

In this notebook, we will use the Diamond price dataset from Kaggle to develop a price prediction model using multiple linear regression. Diamond prices depend on various attributes of the diamond such as carat, cut, clarity etc. The dataset containts over 53000 rows and 10 variables.

<h2> Importing required libraries </h2>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

<h2> Loading the dataset </h2>

In [13]:
print(os.listdir("../"))
diamond = pd.read_csv("../DiamondData/diamonds.csv")
diamond.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


<b>We can see an unwanted column in the dataset, 'Unnamed: 0'. We must drop this column as it plays no role in our prediction.</b>

In [14]:
diamond = diamond.drop('Unnamed: 0', axis = 1)

In [15]:
diamond.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


<h2>Understanding the data better </h2>

In [16]:
diamond.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


<b>Let us now check for null or NaN values in the columns</b>

In [18]:
diamond.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [19]:
diamond.isna().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

<h2> Removing Outliers </h2>

<b>We will now remove the outliers from our dataset by replacing them with NaN values using a simple function.</b>

In [20]:
def outliers(var):
    a = []
    q1 = diamond[var].quantile(0.25)
    q2 = diamond[var].quantile(0.5)
    q3 = diamond[var].quantile(0.75)
    iqr = q3 - q1
    upper_limit = float(q3 + (1.5 * iqr))
    lower_limit = float(q1 - (1.5 * iqr))
    
    for i in diamond[var]:
        if i < lower_limit:
            i = np.NaN
        elif i > upper_limit:
            i = np.NaN
        else:
            i = i
        a.append(i)
    
    return a

for col in diamond.select_dtypes(exclude='object').columns:
    diamond[col] = outliers(col)

In [21]:
diamond.isna().sum()

carat      1889
cut           0
color         0
clarity       0
depth      2545
table       605
price      3540
x            32
y            29
z            49
dtype: int64

<b>Let us replace the NaN values we just added by the mean</b>

In [23]:
for col in diamond.select_dtypes(exclude = 'object').columns:
    diamond[col] = diamond[col].fillna(diamond[col].mean())

In [24]:
diamond.isna().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [25]:
diamond.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.748738,61.776373,57.373404,3159.460833,5.730262,5.732353,3.538265
std,0.394993,1.090063,2.062683,2672.437619,1.116502,1.108833,0.689160
min,0.200000,58.800000,51.600000,326.000000,3.730000,3.680000,1.410000
25%,0.400000,61.200000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.010000,62.500000,59.000000,4478.000000,6.540000,6.540000,4.030000
max,2.000000,64.700000,63.500000,11883.000000,9.260000,9.260000,5.730000


<h2>Categorical variables</h2>

<b>Let us take a closer look at the categorical variables present in the dataset</b>

In [26]:
diamond_categories = diamond.select_dtypes(include = 'object')
diamond_categories['cut'].value_counts()

Ideal        21551
Premium      13791
Very Good    12082
Good          4906
Fair          1610
Name: cut, dtype: int64

In [27]:
diamond_categories['clarity'].value_counts()

SI1     13065
VS2     12258
SI2      9194
VS1      8171
VVS2     5066
VVS1     3655
IF       1790
I1        741
Name: clarity, dtype: int64

In [28]:
diamond_categories['color'].value_counts()

G    11292
E     9797
F     9542
H     8304
D     6775
I     5422
J     2808
Name: color, dtype: int64

<b>Using LabelEncoder from the sklearn library we can encode these categorical variables to numerical variables</b>

In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
diamond_categories = diamond_categories.apply(le.fit_transform)

In [30]:
diamond_categories

,cut,color,clarity
0,2,1,3
1,3,1,2
2,1,1,4
3,3,5,5
4,1,6,3
...,...,...,...
53935,2,0,2
53936,1,0,2
53937,4,0,2
53938,3,4,3


<b>Drop the original categorical variables from the dataset and concat the new (encoded) categorial variables</b>

In [31]:
diamond = diamond.drop(diamond_categories, axis = 1)
diamond = pd.concat([diamond, diamond_categories], axis = 1)

In [34]:
diamond.describe()

,carat,depth,table,price,x,y,z,cut,color,clarity
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.748738,61.776373,57.373404,3159.460833,5.730262,5.732353,3.538265,2.553003,2.594197,3.835150
std,0.394993,1.090063,2.062683,2672.437619,1.116502,1.108833,0.689160,1.027708,1.701105,1.724591
min,0.200000,58.800000,51.600000,326.000000,3.730000,3.680000,1.410000,0.000000,0.000000,0.000000
25%,0.400000,61.200000,56.000000,950.000000,4.710000,4.720000,2.910000,2.000000,1.000000,2.000000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000,2.000000,3.000000,4.000000
75%,1.010000,62.500000,59.000000,4478.000000,6.540000,6.540000,4.030000,3.000000,4.000000,5.000000
max,2.000000,64.700000,63.500000,11883.000000,9.260000,9.260000,5.730000,4.000000,6.000000,7.000000


<h2>Modeling</h2>

<b>We are now moving towards the fun part - Modeling the price function!</b> <br>
We want to model the data in such a way that we can get <b>price</b> as a function of the other variables. That means, we want price to be our 'y', and all other variables to be our 'x1, x2, x3...xn' or 'X'.

In [ ]:
X = diamond.drop('price', axis = 1)
y = diamond['price']

<b>Splitting the data into training and testing datasets</b>

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

<b>Once we have our training and testing datasets, it is time to fit a linear regression model to the data</b>

In [42]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

<b>Let us find the R-squared value for this model</b>

In [43]:
lm.score(X_train,y_train)

0.7013892989783421

In [44]:
lm.coef_

array([4443.09103415,    5.98995467,  -29.62463454, -265.40424351,
        888.99297045, -103.86890453,   38.01745116, -104.03117783,
        148.97468441])

In [48]:
lm.intercept_

-2443.2956084565503

<b>Scoring the model on the test data</b>

In [45]:
lm.score(X_test,y_test)

0.7009832352100172